In [29]:
# Step 1 — Purpose & Overview
#
# This notebook exercises and validates the `process_article_file` function from `cleaner.py`
# on a single example JSON. It is organized into small, numbered steps so you can run them
# one-by-one or end-to-end.
#
# What this notebook is for
# - Quickly test changes to `cleaner.py` on a single file before batch processing.
# - Verify the transformation output structure and a short preview of the content.
# - Troubleshoot common issues (missing imports, missing input file, or bad JSON).
#
# Inputs and outputs
# - Input: a raw article JSON file located in the same folder as this notebook.
# - Output: a transformed JSON written to the `./transformed_articles/` folder.
#
# What you will do
# - Verify imports and environment (Step 2)
# - Define input/output paths and filenames (Step 3)
# - Validate the input and prepare the output folder (Step 4)
# - Run the processing function (Step 5)
# - Verify that the transformed file was produced and is valid JSON (Step 6)
# - (Optional) Compare the original and transformed payloads (Step 7)
# - (Optional) Reset the output file to re-run cleanly (Step 8)
#
# Tip: Run cells top-to-bottom the first time. Afterwards you typically only need
# to tweak Step 3 and re-run Steps 4–6 while iterating on `cleaner.py`.


In [ ]:
# Step 2 — Environment setup & imports
# Sets up the Python environment and imports the processing function.
# - Adds the parent directory to sys.path so `cleaner.py` can be imported.
# - Prints a friendly message showing exactly which file was imported.
# Troubleshooting:
# - If the import fails, confirm `cleaner.py` is in the parent folder or adjust the path.
import sys
import os
import json
from pathlib import Path
from datetime import datetime

# To import from the parent directory, we add it to the system path
# This allows us to import the 'cleaner' module
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

try:
    from cleaner import process_article_file  # type: ignore
    cleaner_path = os.path.join(module_path, 'cleaner.py')
    print(f"✅ Imported 'process_article_file' from: {cleaner_path}")
except Exception as e:
    print("❌ Failed to import from 'cleaner.py'.\n"
          "Make sure 'cleaner.py' exists in the parent folder.")
    raise

print(f"🕒 Run started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📍 Working directory: {Path.cwd()}\n")

✅ Imported 'process_article_file' from: d:\CODE\xai_articles\xai_articles\data_cleaner\test_cleaner\cleaner.py
🕒 Run started: 2025-11-01 13:56:52
📍 Working directory: d:\CODE\xai_articles\xai_articles\data_cleaner\test_cleaner



In [31]:
# Step 3 — Define input/output paths
#
# What this does
# - Sets the base input directory to the current working directory (the notebook's folder).
# - Points the output directory to `./transformed_articles/`.
# - Lets you pick the input filename and the output filename explicitly.
#
# How to use
# - Change `source_file_name` to any JSON sitting next to this notebook.
# - By default `output_file_name` matches the input name; change it if you want a different name.
# - We also remove any previous output file so the verification step reflects a fresh run.


In [ ]:
# Paths
# Set the base directories. `Path.cwd()` should be this notebook's folder when run from VS Code/Jupyter.
base_input_dir = Path.cwd()
# All transformed files will be stored under this subfolder (created in Step 4 if missing).
base_output_dir = base_input_dir / 'transformed_articles'

# Quick Config — set the source and output file names
# Pick the input file to process. Change to try a different article JSON.
source_file_name = 'sample_article.json'  # change this to try another file
# By default we write the output using the same filename; adjust if you want to keep multiple variants.
output_file_name = source_file_name      # or set a different output name if desired

# Resolve full paths used by the cleaner
source_file_path = base_input_dir / source_file_name
output_file_path = base_output_dir / output_file_name

print(f"▶️  Source file: {source_file_path}")
print(f"◀️  Output file: {output_file_path}")

# Remove any residue from previous runs for a clean slate
# This ensures Step 5 writes a brand-new file (some cleaners skip overwriting existing outputs).
if output_file_path.exists():
    output_file_path.unlink()
    print(f"🗑️ Removed previous output: {output_file_path}")

▶️  Source file: d:\CODE\xai_articles\xai_articles\data_cleaner\test_cleaner\2025-01-31_1c6519de55247602e5d9757a4b5e5048.json
◀️  Output file: d:\CODE\xai_articles\xai_articles\data_cleaner\test_cleaner\transformed_articles\2025-01-31_1c6519de55247602e5d9757a4b5e5048.json
🗑️ Removed previous output: d:\CODE\xai_articles\xai_articles\data_cleaner\test_cleaner\transformed_articles\2025-01-31_1c6519de55247602e5d9757a4b5e5048.json


In [19]:
# Step 4 — Validate input and prepare output directory
#
# What this does
# - Fails fast if the chosen input file does not exist (helps catch typos in the filename).
# - Ensures the output directory exists (creates it if needed) so the next step can write the file.
#
# Why it matters
# - Skipping these checks often leads to confusing errors later (e.g., JSON load errors or write failures).
# - Creating the folder here keeps Step 5 focused purely on transformation.


In [20]:
# Ensure input exists
if not source_file_path.exists():
    raise FileNotFoundError(
        f"Input file not found: {source_file_path}. Place a JSON file next to this notebook."
    )

# Ensure output directory exists
base_output_dir.mkdir(parents=True, exist_ok=True)
print(f"📁 Output directory ready: {base_output_dir}")


📁 Output directory ready: d:\CODE\xai_articles\xai_articles\data_cleaner\test_cleaner\transformed_articles


In [21]:
# Step 5 — Run the processing function
#
# What this does
# - Calls `process_article_file(source_file_path, base_input_dir, base_output_dir)` from `cleaner.py`.
# - The function reads the input JSON, cleans and normalizes text (e.g., removes soft hyphens),
#   enriches metadata (word count, reading time), and writes a new JSON into `./transformed_articles/`.
#
# Notes
# - If an output file already exists, the cleaner may skip re-writing to avoid accidental overwrites.
#   This notebook removes the previous output in Step 3 to ensure a fresh run.
# - Any exceptions are printed by the cleaner; Step 6 will also fail if the output wasn't created.


In [22]:
# Run processing
print("🏃 Running process_article_file(...) ...")
# Call the cleaner with:
# - source_file_path: the specific JSON file to transform
# - base_input_dir:   the root path used to compute a relative output path
# - base_output_dir:  the folder where transformed files are written
process_article_file(source_file_path, base_input_dir, base_output_dir)
print("✅ Processing finished.\n")


🏃 Running process_article_file(...) ...
✅ Transformed and saved: d:\CODE\xai_articles\xai_articles\data_cleaner\test_cleaner\transformed_articles\2025-01-31_1c6519de55247602e5d9757a4b5e5048.json
✅ Processing finished.



In [23]:
# Step 6 — Basic verification
#
# Purpose
# - Confirm the transformed file was created in the expected location.
# - Load both original and transformed JSON to ensure they are valid and comparable.
# - Show a compact preview so you can spot-check key fields without dumping everything.
#
# What to look for
# - The output file path printed in Step 3 should now exist.
# - The transformed preview should include `article_id`, `source_info`, `metadata`, `content.article_body`, etc.
# - If the loader fails here, the input may be malformed JSON or the cleaner may have raised an error.


In [24]:
# Verify output file
if not output_file_path.exists():
    raise RuntimeError(
        f"Expected output was not created: {output_file_path}. Check errors above."
    )

# Load original and transformed
with open(source_file_path, 'r', encoding='utf-8') as f:
    original_data = json.load(f)
with open(output_file_path, 'r', encoding='utf-8') as f:
    transformed_data = json.load(f)

# Quick sanity info
orig_keys = list(original_data.keys()) if isinstance(original_data, dict) else None
trans_keys = list(transformed_data.keys()) if isinstance(transformed_data, dict) else None

print("✅ Output JSON loaded successfully.")
print(f"• Original type:   {type(original_data).__name__}")
print(f"• Transformed type:{type(transformed_data).__name__}")
if orig_keys is not None:
    print(f"• Original keys (first 10):   {orig_keys[:10]}")
if trans_keys is not None:
    print(f"• Transformed keys (first 10):{trans_keys[:10]}")

# Preview transformed snippet
# For convenience, build a compact preview without dumping the whole file:
# - If the transformed object is a list/tuple, show only the first element.
# - If it's a dict, show up to the first 10 top-level keys.
preview = transformed_data
if isinstance(transformed_data, (list, tuple)) and len(transformed_data) > 0:
    preview = transformed_data[0]
elif isinstance(transformed_data, dict):
    preview = {k: transformed_data[k] for i, k in enumerate(transformed_data) if i < 10}

print("\n—— Transformed preview ——")
print(json.dumps(preview, indent=2, ensure_ascii=False))


✅ Output JSON loaded successfully.
• Original type:   dict
• Transformed type:dict
• Original keys (first 10):   ['title', 'author', 'content', 'tags', 'categories', 'image', 'retrievedAt', 'source', 'link', 'dateList']
• Transformed keys (first 10):['article_id', 'source_info', 'metadata', 'content', 'entities']

—— Transformed preview ——
{
  "article_id": "2025-01-31_1c6519de55247602e5d9757a4b5e5048",
  "source_info": {
    "source_name": "Dawn",
    "source_link": "https://www.dawn.com/news/1888755/nab-overseas-pakistanis-foundation-join-hands-to-protect-expats-from-fraud",
    "retrieved_at": "2025-10-12T01:04:08.257Z"
  },
  "metadata": {
    "title": "NAB, Overseas Pakistanis Foundation join hands to protect expats from fraud - Business - DAWN.COM",
    "author": null,
    "date_published": "2025-01-31T02:25:27.000Z",
    "image_url": "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVR42mN8Vw8AAmEBb87E6jIAAAAASUVORK5CYII=",
    "categories": [],
    "wo

In [25]:
# Step 7 — (Optional) Compare full original vs transformed
#
# What this does
# - Prints the complete original JSON and the complete transformed JSON for side-by-side inspection.
# - Useful when debugging field mappings or missing values that don't show up in the preview.
#
# Caution
# - For very large inputs, printing the entire payload can be slow and clutter the output.
# - Leave `show_full = False` for routine checks; enable only when you need the full context.


In [26]:
show_full = False  # set to True to print full payloads
if show_full:
    print("================ ORIGINAL DATA ================")
    print(json.dumps(original_data, indent=2, ensure_ascii=False))
    print("\n\n============== TRANSFORMED DATA ==============")
    print(json.dumps(transformed_data, indent=2, ensure_ascii=False))
else:
    print("(Skipped full dump — set 'show_full = True' to print everything.)")


(Skipped full dump — set 'show_full = True' to print everything.)


In [27]:
# Step 8 — (Optional) Reset output
#
# What this does
# - Deletes the generated transformed file so you can re-run Steps 5–6 and see fresh output.
# - Useful when testing changes to `cleaner.py` that would otherwise be skipped if a file already exists.
#
# How to use
# - Set `reset_output = True` to remove the file printed in Step 3.
# - Leave it `False` to keep the artifact for later inspection.


In [28]:
reset_output = False  # set to True to remove the generated file
if reset_output and output_file_path.exists():
    output_file_path.unlink()
    print(f"🧹 Removed: {output_file_path}")
else:
    print("(Output kept — set 'reset_output = True' to remove it.)")

(Output kept — set 'reset_output = True' to remove it.)
